In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [7]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'
DB_NAME = 'grupo8e2'

In [8]:
conn = psycopg2.connect(
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME
)
# Crear un cursor para ejecutar consultas
cursor = conn.cursor()

In [10]:
def cargar_datos(csv_path, table_name):
    conn = None
    cursor = None
    
    try:
        # Conexión con la base de datos
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            name=DB_NAME
        )
        # Crear un cursor para ejecutar consultas
        cursor = conn.cursor()
        
        # Abrir CSV
        with open('../CSV/pedidos.csv', 'r', encoding = 'ISO-8859-1') as file:
            data = file.read().split('\n')
            columns = data[0]
            df = []
            for line in data[1:]:
                df.append(line.split(';'))
        
        # Insertar datos en la tabla
        insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier('Plato'),
            sql.SQL(', ').join(map(sql.Identifier, columns)),
            sql.SQL(', ').join(map(sql.Placeholder, columns))
        )
        # Iterar sobre las filas del DataFrame y ejecutar la consulta de inserción
        for row in df(index=False):
            try:
                data = [value for value in row]
                cursor.execute(insert_query, data)
            except:
                print('f')
        # Confirmar la transacción
        conn.commit()
        print("Data loaded successfully in table {}.".format(table_name))
    except (Exception, Error) as error:
        print("Error loading data:", error)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    # Ruta al archivo CSV 
    ruta_csv = '../CSV/pedidos.csv'

    # Nombre de la tabla PostgreSQL donde se insertarán los datos
    tabla_destino = 'pedidos'

    cargar_datos(ruta_csv, tabla_destino)

Error loading data: invalid dsn: invalid connection option "name"

